## Evaluate the model

In [10]:
import numpy as np

In [11]:
### All random states are assigned to 42 for reproducible results
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [3]:
def get_random_userid(num_users):
    """
    pick 1 random user at random
    Args:
        num_users:: int
    Returns:
        user_id_eval:: int, user_id to evaluate
    """
    user_id_eval = np.random.randint(num_users)
    return user_id_eval
    

In [4]:
def get_y_true(user_id, x_train, y_train):
    # Stacking x_train and y_train
    y_true_tmp = np.hstack((x_train, y_train))
    # keep only the recommended games
    y_true_tmp = y_true_tmp[y_true_tmp[:,-1] == 1]
    # filtering for the user id
    y_true = y_true_tmp[y_true_tmp[:,0] == user_id][:,1]
    return y_true


In [18]:
def get_user_games(user_id, num_games):
    # Generate a list of all games for a specific user
    #games = list(games_mapping.keys())
    games = np.arange(num_games)
    games2pred = np.reshape(games, (-1,1))
    user_game_array_eval = np.hstack(([[user_id]] * len(games), games2pred))
    return user_game_array_eval


In [6]:
def get_y_pred(user_games_array, K):
    preds = model.predict(user_games_array)
    user_games_preds = np.hstack((user_games_array, preds))
    sorted_indices = np.argsort(-user_games_preds[:, -1])
    y_pred = user_games_preds[sorted_indices][:K, :]
    y_pred = y_pred[:,1].astype('int32')
    return y_pred


In [7]:
def eval_k(y_true, y_pred):
    items_relevant_user = len(y_true)
    rec_items_relevant = len(set(y_true).intersection(set(y_pred)))
    recall = rec_items_relevant / items_relevant_user
    precision = rec_items_relevant / K
    return recall, precision


In [8]:
def eval_k_avg(n_users2val, K, num_users, num_games, x_train, y_train):
    """
    Evaluate Top K precision and recall
    Args:
        n_users2val:: int, number of users to evaluate
        K:: int
        num_users:: int, number of users in the test set
        games_mapping:: dict, keys: integer (idx), values: orginal app_id_categorical
        x_train
        y_train

    Returns:
        avg_precision:: float, top K average precision
        avg_recall:: float, top K average recall
    """
    precision = []
    recall = []
    for i in range(n_users2val):
        user_id_eval = get_random_userid(num_users)
        y_true = get_y_true(user_id_eval, x_train, y_train)
        user_games = get_user_games(user_id_eval, num_games)
        y_pred = get_y_pred(user_games, K)
        rcl, prc = eval_k(y_true, y_pred)
        precision.append(prc)
        recall.append(rcl)
    avg_precision = np.mean(precision)
    avg_recall = np.mean(recall)
    return avg_precision, avg_recall


In [17]:
def eval_hit_ratio(n_users2val, K, num_games, x_test):
    counter = 0
    for user_game in x_test[:n_users2val,:]:
        user_games = get_user_games(user_game[0], num_games)
        y_pred = get_y_pred(user_games, K)
        if(user_game[1] in y_pred):
            counter += 1
    return counter / n_users2val
